<a href="https://colab.research.google.com/github/caiocezar400/DeepSea-Solutions/blob/main/Din%C3%A2mica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# ---------------------------------------------------------
# 1. FUNÇÕES DE CINEMÁTICA (FK & JACOBIANO)
# ---------------------------------------------------------

def tr2rpy(T):
    """
    Extrai os ângulos de Euler (Roll, Pitch, Yaw) de uma Matriz de Transformação Homogênea 4x4.
    Convenção utilizada: ZYX (Yaw-Pitch-Roll).
    """
    R = T[:3, :3]
    sy = np.sqrt(R[0, 0]**2 + R[1, 0]**2)
    singular = sy < 1e-6

    if not singular:
        roll  = np.arctan2(R[2, 1], R[2, 2])
        pitch = np.arctan2(-R[2, 0], sy)
        yaw   = np.arctan2(R[1, 0], R[0, 0])
    else:
        roll  = np.arctan2(-R[1, 2], R[1, 1])
        pitch = np.arctan2(-R[2, 0], sy)
        yaw   = 0

    return np.array([roll, pitch, yaw])

def get_tf_mat(a, d, alpha, theta):
    """
    Retorna a Matriz de Transformação Homogênea 4x4 (DH Padrão).
    """
    ct, st = np.cos(theta), np.sin(theta)
    ca, sa = np.cos(alpha), np.sin(alpha)
    return np.array([
        [ct, -st*ca, st*sa, a*ct],
        [st, ct*ca, -ct*sa, a*st],
        [0,  sa,     ca,     d   ],
        [0,  0,      0,      1   ]
    ])

def forward_kinematics(links, q):
    """
    Calcula a posição do efetuador final (x, y, z).
    Retorna: pos (vetor 3x1), transforms (lista de matrizes 4x4)
    """
    T = np.eye(4)
    transforms = []

    for i, link in enumerate(links):
        a = link['a']
        alpha = link['alpha']

        if link['type'] == 'R':
            d = link['d']
            theta = q[i] + link['offset']
        else:
            d = q[i] + link['offset']
            theta = link['theta_const']

        Ti = get_tf_mat(a, d, alpha, theta)
        T = T @ Ti
        transforms.append(T)

    return T[:3, 3], transforms

def get_jacobian_numerical_6dof(links, q, delta=1e-5):
    """
    Calcula o Jacobiano 6xN completo (Posição + Orientação).
    """
    n = len(q)
    J = np.zeros((6, n))
    _, transforms = forward_kinematics(links, q)
    T_curr = transforms[-1]

    pos_curr = T_curr[:3, 3]
    rpy_curr = tr2rpy(T_curr)

    for i in range(n):
        q_perturbed = q.copy()
        q_perturbed[i] += delta

        _, transforms_new = forward_kinematics(links, q_perturbed)
        T_new = transforms_new[-1]

        pos_new = T_new[:3, 3]
        rpy_new = tr2rpy(T_new)

        diff_linear = (pos_new - pos_curr) / delta
        diff_angular_raw = rpy_new - rpy_curr
        diff_angular_norm = (diff_angular_raw + np.pi) % (2 * np.pi) - np.pi
        diff_angular = diff_angular_norm / delta

        J[:3, i] = diff_linear
        J[3:, i] = diff_angular

    return J

# ---------------------------------------------------------
# 2. DINÂMICA (ALGORITMO NEWTON-EULER RECURSIVO)
# ---------------------------------------------------------

def recursive_newton_euler(links, q, qd, qdd, g_vec=np.array([0, -9.81, 0])):
    """
    Calcula a Dinâmica Inversa (Torque).
    """
    n = len(links)
    omega = [np.zeros(3)] * (n + 1)
    d_omega = [np.zeros(3)] * (n + 1)
    d_v = [np.zeros(3)] * (n + 1)
    d_v[0] = -g_vec
    z0 = np.array([0, 0, 1])

    Rs = []
    Ps = []

    # --- Passo Avante ---
    for i in range(n):
        l = links[i]
        a, alpha = l['a'], l['alpha']

        if l['type'] == 'R':
            d = l['d']
            theta = q[i] + l['offset']
            var_qd, var_qdd = qd[i], qdd[i]
            d_rel, dd_rel = 0, 0
        else:
            d = q[i] + l['offset']
            theta = l['theta_const']
            var_qd, var_qdd = 0, 0
            d_rel, dd_rel = qd[i], qdd[i]

        ct, st = np.cos(theta), np.sin(theta)
        ca, sa = np.cos(alpha), np.sin(alpha)

        R_im1_i = np.array([[ct, -st*ca, st*sa], [st, ct*ca, -ct*sa], [0, sa, ca]])
        R_i_im1 = R_im1_i.T
        Rs.append(R_i_im1)

        P_in_i = R_i_im1 @ np.array([a*ct, a*st, d])
        Ps.append(P_in_i)

        if l['type'] == 'R':
            omega[i+1] = R_i_im1 @ omega[i] + var_qd * z0
            d_omega[i+1] = R_i_im1 @ d_omega[i] + np.cross(R_i_im1 @ omega[i], var_qd * z0) + var_qdd * z0
            d_v[i+1] = R_i_im1 @ d_v[i] + np.cross(d_omega[i+1], P_in_i) + np.cross(omega[i+1], np.cross(omega[i+1], P_in_i))
        else:
            omega[i+1] = R_i_im1 @ omega[i]
            d_omega[i+1] = R_i_im1 @ d_omega[i]
            d_v[i+1] = R_i_im1 @ (d_v[i] + 2*np.cross(omega[i], z0*d_rel) + z0*dd_rel) + \
                       np.cross(d_omega[i+1], P_in_i) + np.cross(omega[i+1], np.cross(omega[i+1], P_in_i))

    # --- Passo Atrás ---
    F = [np.zeros(3)] * (n + 1)
    N = [np.zeros(3)] * (n + 1)
    f = [np.zeros(3)] * (n + 1)
    n_mom = [np.zeros(3)] * (n + 1)
    tau = np.zeros(n)

    for i in range(n-1, -1, -1):
        m = links[i]['m']
        c = links[i]['cm']
        I = links[i]['I']

        d_vc = d_v[i+1] + np.cross(d_omega[i+1], c) + np.cross(omega[i+1], np.cross(omega[i+1], c))
        F[i+1] = m * d_vc
        N[i+1] = I @ d_omega[i+1] + np.cross(omega[i+1], I @ omega[i+1])

        if i == n - 1:
            f_next, n_next = np.zeros(3), np.zeros(3)
            R_next_curr = np.eye(3)
            term_torque = np.zeros(3)
        else:
            f_next, n_next = f[i+2], n_mom[i+2]
            R_next_curr = Rs[i+1].T
            P_next_in_curr = Rs[i+1].T @ Ps[i+1]
            term_torque = np.cross(P_next_in_curr, R_next_curr @ f_next)

        f[i+1] = R_next_curr @ f_next + F[i+1]
        n_mom[i+1] = N[i+1] + R_next_curr @ n_next + np.cross(c, F[i+1]) + term_torque

        n_in_prev = Rs[i] @ n_mom[i+1]
        f_in_prev = Rs[i] @ f[i+1]

        if links[i]['type'] == 'R':
            tau[i] = n_in_prev[2]
        else:
            tau[i] = f_in_prev[2]

    return tau

# ---------------------------------------------------------
# 3. INTERFACE E CONFIGURAÇÃO DO USUÁRIO
# ---------------------------------------------------------

def get_prebuilt_robot():
    """
    Retorna um robô antropomórfico (6-DOF) pré-configurado.
    Similar a um braço PUMA.
    """
    print("\nCarregando Robô Pré-construído (6-DOF Antropomórfico)...")

    # Propriedades de massa genéricas
    I_1 = np.diag([0.01, 0.01, 0.01])
    I_2 = np.diag([0.01, 0.02, 0.01])
    I_3 = np.diag([0.01, 0.01, 0.03])
    I_4 = np.diag([0.04, 0.01, 0.01])

    links = [
        # Junta 1 (Base)
        {
            'type': 'R', 'a': 0, 'alpha': np.pi/2, 'd': 0.5, 'offset': 0, 'theta_const': 0,
            'm': 4.0, 'cm': np.array([0, 0, 0]), 'I': I_1
        },
        # Junta 2
        {
            'type': 'R', 'a': 0.8, 'alpha': 0, 'd': 0, 'offset': np.pi/2, 'theta_const': 0,
            'm': 3.0, 'cm': np.array([0.4, 0, 0]), 'I': I_2
        },
        # Junta 3
        {
            'type': 'R', 'a': 0.6, 'alpha': 0, 'd': 0, 'offset': 0, 'theta_const': 0,
            'm': 2.0, 'cm': np.array([0.3, 0, 0]), 'I': I_3
        },
        # Junta 4
        {
            'type': 'P', 'a': 0.6, 'alpha': 0, 'd': 0, 'offset': 0, 'theta_const': 0,
            'm': 2.0, 'cm': np.array([0.3, 0, 0]), 'I': I_4
        }
    ]
    return links

def get_manual_robot_parameters():
    """
    Solicita os parâmetros ao usuário manualmente.
    """
    print("--- Configuração Manual do Robô ---")
    config_str = input("Digite a configuração das juntas (ex: RR, RRP, RRR): ").upper().strip()
    if not config_str: config_str = "RR"

    links = []
    print(f"\nDefinindo {len(config_str)} juntas (Parâmetros DH Padrão)...")

    for i, char in enumerate(config_str):
        tipo_junta = 'Rotacional' if char=='R' else 'Prismática'
        print(f"\nJunta {i+1} ({tipo_junta}):")

        a = float(input(f"  a (Comprimento do Elo) [1.0]: ") or 1.0)
        alpha = float(input(f"  alpha (Torção em rad) [0.0]: ") or 0.0)

        if char == 'R':
            d = float(input(f"  d (Deslocamento do Elo) [0.0]: ") or 0.0)
            offset = float(input(f"  theta offset (rad) [0.0]: ") or 0.0)
            theta_const = 0.0
        else:
            theta_const = float(input(f"  theta (Rotação fixa) [0.0]: ") or 0.0)
            offset = float(input(f"  d offset (m) [0.0]: ") or 0.0)
            d = 0.0

        m = float(input(f"  Massa (kg) [1.0]: ") or 1.0)
        print("  Tensor de Inércia (Diagonal simplificada Ix, Iy, Iz):")
        ix = float(input("    Ix [0.1]: ") or 0.1)
        iy = float(input("    Iy [0.1]: ") or 0.1)
        iz = float(input("    Iz [0.1]: ") or 0.1)

        links.append({
            'type': char, 'a': a, 'alpha': alpha, 'd': d,
            'offset': offset, 'theta_const': theta_const,
            'm': m, 'cm': np.array([a/2, 0, 0]),
            'I': np.diag([ix, iy, iz])
        })

    return links

def generate_task_trajectory(traj_type, t_arr, center_pos):
    """Gera Posição (X, Y, Z) e Velocidade (V) Cartesianas desejadas."""
    positions = []
    velocities = []

    radius = 0.5
    omega = 2.0 * np.pi / t_arr[-1]

    for t in t_arr:
        if traj_type == '1': # Linha Reta
            start = center_pos
            end = center_pos + np.array([0.5, 0.5, 0])
            s = t / t_arr[-1]
            pos = start + s * (end - start)
            vel = (end - start) / t_arr[-1]

        elif traj_type == '2': # Círculo
            pos = center_pos + np.array([
                radius * np.cos(omega * t) - radius,
                radius * np.sin(omega * t),
                0
            ])
            vel = np.array([
                -radius * omega * np.sin(omega * t),
                radius * omega * np.cos(omega * t),
                0
            ])
        elif traj_type == '3': # Semicírculo
            omega_semi = omega / 2.0
            pos = center_pos + np.array([
                radius * np.cos(omega_semi * t) - radius,
                radius * np.sin(omega_semi * t),
                0
            ])
            vel = np.array([
                -radius * omega_semi * np.sin(omega_semi * t),
                radius * omega_semi * np.cos(omega_semi * t),
                0
            ])

        elif traj_type == '4': # Figura 8
            scale = 0.3
            pos = center_pos + np.array([
                scale * np.sin(omega * t),
                scale * np.sin(2 * omega * t),
                0
            ])
            vel = np.array([
                scale * omega * np.cos(omega * t),
                scale * 2 * omega * np.cos(2 * omega * t),
                0
            ])
        else:
            pos = center_pos
            vel = np.zeros(3)

        positions.append(pos)
        velocities.append(vel)

    return np.array(positions), np.array(velocities)

# ---------------------------------------------------------
# 4. LOOP PRINCIPAL DE SIMULAÇÃO
# ---------------------------------------------------------

def main():
    print("========================================")
    print("   SIMULADOR DE DINÂMICA ROBÓTICA")
    print("========================================")

    # 1. Seleção do Modo de Robô
    print("\nEscolha o Robô:")
    print("1. Robô Pré-construído (6-DOF Antropomórfico)")
    print("2. Definir Manualmente (Entrada de Parâmetros)")

    robot_mode = input("Opção [1]: ").strip() or '1'

    if robot_mode == '1':
        links = get_prebuilt_robot()
    else:
        links = get_manual_robot_parameters()

    n = len(links)

    # 2. Configurar Tempo
    T_duration = 5.0
    dt = 0.05
    time_steps = np.arange(0, T_duration, dt)

    # 3. Escolher Trajetória
    print("\nSelecione a Trajetória (Espaço da Tarefa/Cartesiano):")
    print("1. Linha Reta")
    print("2. Círculo")
    print("3. Semi-círculo")
    print("4. Figura 8")
    traj_choice = input("Escolha [1]: ").strip() or '1'

    # Configuração Inicial (Home)
    q_curr = np.zeros(n)
    if n >= 2: q_curr[1] = 0.1

    start_pos, start_transforms = forward_kinematics(links, q_curr)
    start_rpy = tr2rpy(start_transforms[-1])

    # Gerar Caminho Alvo
    X_d_pos, V_d_lin = generate_task_trajectory(traj_choice, time_steps, start_pos)

    history_q, history_qd, history_qdd, history_tau = [], [], [], []
    qd_curr = np.zeros(n)

    print("\nSimulando Rastreamento 6-DoF...")

    for i, t in enumerate(time_steps):
        # --- 1. Estado Atual ---
        x_curr_pos, transforms = forward_kinematics(links, q_curr)
        x_curr_rpy = tr2rpy(transforms[-1])

        # --- 2. Calcular Erros ---
        err_linear = X_d_pos[i] - x_curr_pos
        err_angular = start_rpy - x_curr_rpy
        err_angular = (err_angular + np.pi) % (2 * np.pi) - np.pi
        err_total = np.concatenate((err_linear, err_angular))

        # --- 3. Calcular Jacobiano ---
        J = get_jacobian_numerical_6dof(links, q_curr)

        # --- 4. Controle ---
        Kp = 5.0
        v_desired = np.concatenate((V_d_lin[i], np.zeros(3)))
        v_cmd = v_desired + Kp * err_total

        # --- 5. Cinemática Inversa Diferencial ---
        lamb = 0.01
        J_inv = J.T @ np.linalg.inv(J @ J.T + lamb**2 * np.eye(6))
        qd_new = J_inv @ v_cmd

        # --- 6. Integração ---
        q_next = q_curr + qd_new * dt

        # --- 7. Dinâmica ---
        qdd_curr = (qd_new - qd_curr) / dt
        tau = recursive_newton_euler(links, q_curr, qd_new, qdd_curr)

        history_q.append(q_curr)
        history_qd.append(qd_new)
        history_qdd.append(qdd_curr)
        history_tau.append(tau)

        q_curr = q_next
        qd_curr = qd_new

    # Plotagem
    H_q = np.array(history_q)
    H_qd = np.array(history_qd)
    H_qdd = np.array(history_qdd)
    H_tau = np.array(history_tau)

    fig, axs = plt.subplots(2, 2, figsize=(12, 10))
    fig.suptitle(f'Dinâmica do Manipulador - Trajetória Tipo {traj_choice}', fontsize=16)

    labels = [f'Junta {k+1}' for k in range(n)]

    for k in range(n): axs[0,0].plot(time_steps, H_q[:, k], label=labels[k])
    axs[0,0].set_title('Posição das Juntas (rad ou m)'); axs[0,0].grid(True); axs[0,0].legend()

    for k in range(n): axs[0,1].plot(time_steps, H_qd[:, k], label=labels[k])
    axs[0,1].set_title('Velocidade das Juntas'); axs[0,1].grid(True)

    for k in range(n): axs[1,0].plot(time_steps, H_qdd[:, k], label=labels[k])
    axs[1,0].set_title('Aceleração das Juntas'); axs[1,0].grid(True)

    for k in range(n): axs[1,1].plot(time_steps, H_tau[:, k], label=labels[k])
    axs[1,1].set_title('Torque / Força (N.m ou N)'); axs[1,1].grid(True)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    print("Gerando gráficos...")
    plt.show()

if __name__ == "__main__":
    main()